In [10]:
import pandas as pd
import numpy as np

import yfinance as yf


In [12]:
def esg_score_weight(data: pd.DataFrame, weights: np.array):
    """
    
    :param: 
    :param:
    :param: 
    :returns:
    """
    if np.sum(weights) != 1:
        return("Weights must sum to 1")
    else:


        data["weighted_score"] = (data["environment_score"] * (weights[0]/1)) + \
                           (data["governance_score"] * (weights[1]/1)) + \
                           (data["social_score"] * (weights[2]/1))
        for index, row in data.iterrows():
            if row['CurrencyCode'] == 'SEK':
                data.at[index, 'stock_symbol'] += '.ST'
            elif row['CurrencyCode'] == 'NOK':
                data.at[index, 'stock_symbol'] += '.OL'
            elif row['CurrencyCode'] == 'DKK':
                data.at[index, 'stock_symbol'] += '.CO'
        return data


In [13]:
df1 = esg_score_weight(pd.read_excel(r'DATA/ESG_DATA.xlsx'),np.array([1,0,0]))

In [14]:
df1

,Isin,CurrencyCode,company_name,environment_grade,environment_level,environment_score,governance_grade,governance_level,governance_score,social_grade,social_level,social_score,stock_symbol,total,total_grade,total_level,weighted_score
0,SE0011337708,SEK,AAK AB (publ.),AA,Excellent,647,BB,Medium,345,BB,Medium,307,AAK.ST,1299,A,High,647.0
1,CH0012221716,SEK,ABB Ltd,BB,Medium,306,B,Medium,220,BBB,High,425,ABB.ST,951,BBB,High,306.0
2,SE0007897079,SEK,ACADIA Pharmaceuticals Inc.,B,Medium,290,BB,Medium,300,BB,Medium,344,ACAD.ST,934,BBB,High,290.0
3,SE0014781795,SEK,Addtech AB (publ.),A,High,500,BB,Medium,300,BB,Medium,300,ADDT-B.ST,1100,BBB,High,500.0
4,CA00830W1059,SEK,AEC Securities Public Company Limited,B,Medium,240,BB,Medium,300,BB,Medium,300,AEC.ST,840,BB,Medium,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,DK0060036564,DKK,Spar Nord Bank A/S,B,Medium,230,BB,Medium,300,B,Medium,216,SPNO.CO,746,B,Medium,230.0
283,DK0060696300,DKK,Straker Translations Limited,BBB,High,410,BB,Medium,315,BB,Medium,301,STG.CO,1026,BBB,High,410.0
284,DK0010311471,DKK,Sydbank A/S,A,High,500,BB,Medium,300,BB,Medium,301,SYDB.CO,1101,BBB,High,500.0
285,DK0060477503,DKK,Thorney Opportunities Ltd,B,Medium,215,BB,Medium,305,B,Medium,203,TOP.CO,723,B,Medium,215.0


In [5]:
import yfinance as yf
import pandas as pd




#Function that takes a dataframe with the layout as above as well as a timeframe in which you want your data from 

def monthly_close(data: pd.DataFrame, dates: tuple):
    symbols = data['stock_symbol'].unique()

# create a new dataframe to store the monthly closing data
    monthly_close_df = pd.DataFrame()
    for symbol in symbols:
    # retrieve data from yfinance
        stock_data = yf.download(symbol, start=dates[0], end=dates[1], interval='1mo', progress=False)
        
        # extract the 'Close' column and rename it with the stock symbol
        stock_data = stock_data[['Close']].rename(columns={'Close': symbol})
        
        # add the weighted score for the stock
        weighted_score = data.loc[data['stock_symbol']==symbol, 'weighted_score'].iloc[0]
        stock_data[symbol + '_weighted'] = weighted_score
        
        # append the stock data to the monthly_close_df
        monthly_close_df = pd.concat([monthly_close_df, stock_data], axis=1)
    return(monthly_close_df.dropna(axis=1,how='all'))    
        



In [6]:
df2 = monthly_close(df1,dates=('2010-01-01','2022-03-01'))


1 Failed download:
- COLL.ST: Data doesn't exist for startDate = 1262300400, endDate = 1646089200

1 Failed download:
- JOBS.ST: Data doesn't exist for startDate = 1262300400, endDate = 1646089200

1 Failed download:
- MSEIS.OL: No timezone found, symbol may be delisted

1 Failed download:
- SDRL.OL: Data doesn't exist for startDate = 1262300400, endDate = 1646089200


.to_csv('cleaned_data.csv', index=False)

In [11]:
print(df2)
df2.to_csv('df.csv') 

                               AAK.ST  AAK.ST_weighted      ABB.ST  \
Date                                                                 
2009-12-31 23:00:00+00:00   28.250000           1299.0  135.399994   
2010-01-31 23:00:00+00:00   31.666666           1299.0  144.000000   
2010-02-28 23:00:00+00:00   32.666664           1299.0  157.899994   
2010-03-31 22:00:00+00:00   30.500000           1299.0  140.899994   
2010-04-30 22:00:00+00:00   23.750000           1299.0  135.199997   
...                               ...              ...         ...   
2021-09-30 22:00:00+00:00  187.699997           1299.0  283.899994   
2021-10-31 23:00:00+00:00  181.149994           1299.0  312.299988   
2021-11-30 23:00:00+00:00  195.300003           1299.0  345.399994   
2021-12-31 23:00:00+00:00  173.850006           1299.0  318.799988   
2022-01-31 23:00:00+00:00  172.949997           1299.0  320.399994   

                           ABB.ST_weighted    ACAD.ST  ACAD.ST_weighted  \
Date          

In [12]:
df2.to_csv('df.csv')